In [1]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [3]:
import tensorflow as tf

from modules.models import Model_with_dropout_freeze_model_3
from modules.utils import get_dataset

from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, TensorBoard

2023-04-05 17:00:49.421643: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from modules.dataset import LABELS

In [5]:
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
EXPERIMENT_NAME = "under_sampling_5_folds_with_cross_entropy_loss_freeze_imagenet_model_3"

In [6]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def __init__(self, fold_num):
        self.fold_num = fold_num
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling_5_folds_dataset/folds/fold{self.fold_num}/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling_5_folds_dataset/folds/fold{self.fold_num}/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## tf.keras.mixed_precision for faster training

In [7]:
print(tf.keras.mixed_precision.global_policy())
# tf.keras.mixed_precision.set_global_policy('mixed_float16')
print(tf.keras.mixed_precision.global_policy())

<Policy "float32">
<Policy "float32">


## Model Training

In [8]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/Under_sampling-multilabel_classification'

In [9]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [10]:
import datetime

def get_callbacks(NAME, weight_option, fold_num):
    log_dir = f"{CURRENT_PATH}/logs/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model_checkpoint_callback = ModelCheckpoint(f'results/models/{EXPERIMENT_NAME}/{NAME}_{weight_option}_FOLD_{fold_num}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=10, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback, tensorboard_callback

In [11]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100

In [12]:
import pprint
import numpy as np

def check_label_on_dataset(train_dataset, test_dataset):
    y_list = []
    for x,y in train_dataset:
        # print(y)
        y_list.append(y)
    train_labels = np.vstack(y_list)

    y_list = []
    for x,y in test_dataset:
        # print(y)
        y_list.append(y)
    test_labels = np.vstack(y_list)

    # Print
    pprint.pprint(list(zip(LABELS,train_labels.sum(axis=0))))
    print("Count:", len(train_labels))
    pprint.pprint(list(zip(LABELS, test_labels.sum(axis=0))))
    print("Count: ", len(test_labels))

    print("All:", len(train_labels)+len(test_labels))

In [13]:
# TODO: Main
import time
from tqdm.notebook import tqdm

NUM_FOLDS = 5
weight_option = 'imagenet' # use `imagenet` or `None` only

# Fold [1, 3]
for fold_num in tqdm(range(1, NUM_FOLDS + 1)):
    
    # Callbacks
    (
        model_checkpoint_callback, 
        early_stop_callback, 
        reduce_lr_callback, 
        lr_logging_callback, 
        tensorboard_callback
    ) = get_callbacks(
        NAME, weight_option, fold_num
    )

    # Path for CSV
    path = os.path.join(CURRENT_PATH, "results", "history", EXPERIMENT_NAME, f"{NAME}_{weight_option}_FOLD_{fold_num}")
    os.makedirs(path, exist_ok=True)

    # CSV Logger
    csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

    # Dataset
    dataset = Dataset(fold_num)
    train_dataset, test_dataset = dataset.get_train(), dataset.get_test()
    # check_label_on_dataset(train_dataset, test_dataset)

    # Modeling
    transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False, 
        weights=weight_option,
        input_shape=(224, 224, 3),
        pooling=None
    )
    transfer_model.trainable = False

    model = Model_with_dropout_freeze_model_3(
        transfer_model,
    )
    model = model.get_model()
    model._name = f"{EXPERIMENT_NAME}_FOLD_{fold_num}"
    model.summary()

    # Record time for training
    start = time.time()
    
    # Visualize
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        verbose=1, # Show Progress Bar while Traning
        callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
    )
    
    end = time.time()
    print(
        "===== " * 5,
        "Model Traninig for: {:.2f} second(s)".format(end - start),
        "===== " * 5,
        "\n\n"
    )
    
    # break

  0%|          | 0/5 [00:00<?, ?it/s]

2023-04-05 17:00:52.574207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 17:00:53.333526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8011 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:31:00.0, compute capability: 8.0


Model: "under_sampling_5_folds_with_cross_entropy_loss_freeze_imagenet_model_3_FOLD_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                

2023-04-05 17:01:05.044419: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-05 17:01:05.948825: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-05 17:01:05.950563: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-05 17:01:05.950621: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-04-05 17:01:05.951833: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-05 17:01:05.951960: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      1/Unknown - 10s 10s/step - loss: 0.7104 - auc: 0.3127 - f1_score: 0.0000e+00 - precision: 0.0988 - recall: 0.7273

2023-04-05 17:01:07.196717: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3066/3066 [==============================] - 183s 56ms/step - loss: 0.2909 - auc: 0.4976 - f1_score: 0.0366 - precision: 0.1269 - recall: 0.0027 - val_loss: 0.2648 - val_auc: 0.5000 - val_f1_score: 0.0200 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/100
3066/3066 [==============================] - 169s 55ms/step - loss: 0.2866 - auc: 0.4977 - f1_score: 0.0327 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.2588 - val_auc: 0.5000 - val_f1_score: 0.0200 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 3/100
3066/3066 [==============================] - 169s 55ms/step - loss: 0.2858 - auc: 0.4970 - f1_score: 0.0327 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.2576 - val_auc: 0.5000 - val_f1_score: 0.0200 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 4/100
3066/3066 [==============================] - 169s 55ms/step - loss: 0.2855 - auc: 0.5007 - f1_score: 0.0327 - precision: 0.0000e+00 - r

In [14]:
print("Done")

Done
